# Practise YOLOv5

## Import Libraries

In [ ]:
# Libraries to load and analyze data
import os
import tarfile
import urllib.request
import glob
import numpy as np
import shutil
import pandas as pd
import xml.etree.ElementTree as ET
from sklearn import preprocessing
from time import time
from tqdm import tqdm

# Libraries to work with images and build models
import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageDraw
from albumentations import (HorizontalFlip, ShiftScaleRotate, VerticalFlip, Normalize,Flip,
                            Compose, GaussNoise)
import cv2
import shutil
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import display

device = 'cpu'
print(f'Using device: {device}')

dataset_path = 'data/VOCdevkit/VOC2012/'

## Load Annotations

Loading only annotations for the images made in the year 2010 as pandas DataFrame.

In [ ]:
def xml_to_csv(path=dataset_path+'Annotations/'):
    xml_list = []
    for xml_file in tqdm(glob.glob(path + '/2010*.xml')):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for obj in root.findall('object'):
            bbx = obj.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = obj.find('name').text

            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[2].text),
                     int(root.find('size')[1].text),
                     label,
                     xmin,
                     ymin,
                     xmax,
                     ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'channels', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [ ]:
xml_df = xml_to_csv()
xml_df.to_csv('VOC_2010.csv', index=None)
print('Saved as csv file.')

For easier troubleshooting I keep only the top 100 objects.

In [ ]:
xml_df = xml_df[:100]

In [ ]:
xml_df.head()

In [ ]:
print(f'Oveall number of objects: {xml_df.shape[0]}')
print(f'Number of images in the dataframe: {len(np.unique(xml_df["filename"]))}')

## Data Preparation

Encode object classes with `LabelEncoder`.

In [ ]:
le = preprocessing.LabelEncoder()
xml_df['class'] = le.fit_transform(xml_df['class'])
xml_df.head()

Save the encoding as dictionary in the `mappring` variable

In [ ]:
class_to_id_mapping = dict(zip(le.classes_, range(len(le.classes_))))
class_to_id_mapping

Now we need to convert our data to the **YOLOv5** format.

The annotation for each object should contain the following information:

`Class X Y Width Height`

For this, we'll use the helper function `convert_to_yolov5`.

In [ ]:
def convert_to_yolov5(df, unique_img_names):
    df_array = np.array(df)

    # For each unique image name
    for unique_img_name in unique_img_names:
        print_buffer = []

        # For each object
        for i in range(len(df)):
            point =  df_array[i]
            img_name = point[0]

            if img_name == unique_img_name:
                class_id = point[4]

                # Transform bbox coordinates
                X = (point[5] + point[7]) / 2
                Y = (point[6] + point[8]) / 2
                Width = (point[7] - point[5])
                Height = (point[8] - point[6])

                # Normalize coordinates
                image_w, image_h = point[2], point[3]
                X /= image_w
                Y /= image_h
                Width /= image_w
                Height /= image_h
              
                # Save the bbox information to the buffer
                print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, X, Y, Width, Height))

        # Get file names to save the annotation
        save_file_name = os.path.join(dataset_path+'JPEGImages/yolov5/labels/', unique_img_name.replace("jpg", "txt"))
        # Save the annotations to disk
        print("\n".join(print_buffer), file=open(save_file_name, "w"))
        # Copy the image to the new folder                
        shutil.copy(dataset_path+'JPEGImages/'+unique_img_name, dataset_path+'JPEGImages/yolov5/images/'+unique_img_name)


In [ ]:
# First let's get the unique image names
unique_img_names = xml_df['filename'].unique()
print('Unique image names number: ', len(unique_img_names))

In [ ]:
# Then we convert the annotations to YOLOv5 format
convert_to_yolov5(xml_df, unique_img_names)

Let's test the transformed annotations via a viaualization.

In [ ]:
id_to_class_mapping = dict(zip(class_to_id_mapping.values(), class_to_id_mapping.keys()))
id_to_class_mapping

In [ ]:
def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size

    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)), outline="#FF0000FF", width=2)

        plotted_image.text((x0, y0 - 10), id_to_class_mapping[(int(obj_cls))], fill="#000")
    
    
    plt.imshow(np.array(image))
    plt.show()

In [ ]:
# Pick a random annotation file 
annotation_random = !ls {dataset_path}/JPEGImages/yolov5/*txt | gshuf -n 1
annotation_random

In [ ]:
# Load random annotation file
annotation_file = annotation_random[0]
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

#Get the corresponding image file
image_file = annotation_file.replace("txt", "jpg")
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)

Trained model is available in the `hub` class of the `torch` library.

Let's see what would be the model predictions for our random image.

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.model.to(device)
print(next(model.model.parameters()).device)
img = annotation_random[0].replace("txt", "jpg")

# Inference
results = model(img)

# Results
results.print()

## Train YOLOv5

Train the **yolov5** model using the prepared `train.py` file from the repository https://github.com/ultralytics/yolov5.git.

For this model, images and object classes should be located in the images and labels folders, respectively.

Test the model on the validation set and display the resulting images with bbox and object classes.

I will use the `coco128` dataset and a predefined YOLOv5 Medium size model with weights.

Not to overload the machine, I am using a small batch of 4 images.

In [ ]:
!python train.py --img 640 --batch 4 --epochs 10 --data coco128.yaml --weights yolov5m.pt --cache

The model has achieved the mAP=0.86 after 10 training epochs, which is a very good result.

Here is a summary of metrics for training and validation:
![Training and validation results](results.png)

Here some images with boxes and class labels from the validation process:

Image 1
![Image 1](val_batch0_pred.jpg)

Image 2
![Image 2](val_batch1_pred.jpg)

Image 3
![Image 3](val_batch2_pred.jpg)
